# Index

In [1]:
import fitz
from langchain_text_splitters import RecursiveCharacterTextSplitter
# This is only for generic texts - There are better ways to Document and index PDFs
from utils import *
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

In [2]:
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.0)
PDF = "temp.pdf"

In [3]:
pdf = fitz.open(PDF)
full_text = get_all_text_content(pdf)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
docs = splitter.create_documents([full_text])

In [4]:
docs = add_meta(docs, {"pdf_url" : PDF})

In [5]:
db = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db", collection_name='rag-q_a')

In [8]:
all_docs_meta = db.get()['metadatas']
pdf_indexed = []
for item in all_docs_meta:
    pdf_indexed.append(item['pdf_url'])
if PDF in pdf_indexed:
    print("No indexing")
    pass
else:
    print("Indexing to DB")
    db.add_documents(docs)

No indexing


In [9]:
retriever = db.as_retriever(search_kwargs={'filter': {'pdf_url':PDF}})

# Retrieve from DB

In [10]:
query = "Elaborate on what BERT is. I am new to this concepts"

In [11]:
retrieved_docs = retriever.invoke(query)

In [12]:
context = format_docs(retrieved_docs)

In [13]:
template = [
        ('system', "From the given CONTEXT from a paper, answer the given QUERY."),
        ('user', f"CONTEXT: {context}"),
        ('user', f"QUERY: {query}")
    ]
response = model.invoke(template)